# Diamond Pricing Analysis
## Authored by Trinity Gahagen
#### Phase 3 Project for Flatiron School
---
---

# Table of Contents
1. [Data Overview](#data-overview)
    1. [Inital Data Exploration](#initial-data-exploration)
2. [EDA: Descriptive Analysis](#eda-descriptive-analysis)

# Data Overview
---

*Dataset Kaggle Page:* https://www.kaggle.com/datasets/miguelcorraljr/brilliant-diamonds

This dataset was scraped back in 2020 off of Brilliant Earth's diamond catalogue. It hosts the folllowing columns:

|Attribute|	Description	|Data Type|
|---------|-------------|---------|
|id|Diamond identification number provided by Brilliant Earth|int
|url|URL for the diamond details page|string
|shape|External geometric appearance of a diamond|string/categorical
|price|Price in U.S. dollars|int
|carat|Unit of measurement used to describe the weight of a diamond|float
|cut|Facets, symmetry, and reflective qualities of a diamond|string/categorical
|color|Natural color or lack of color visible within a diamond, based on the GIA grade scale|string/categorical
|clarity|Visibility of natural microscopic inclusions and imperfections within a diamond|string/categorical
|report|Diamond certificate or grading report provided by an independent gemology lab|string
|type|Natural or lab created diamonds|string
|date_fetched|Date the data was fetched|date

### Initial Data Exploration

In [1]:
# Base Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical modules
import scipy.stats as stats

# ML Tools
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV, cross_val_score
# Classification metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, roc_auc_score
# Regression metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, explained_variance_score

# ML Models
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# EDA: Descriptive Analysis
---